## 1. Raw Data preprocess

First, I transform the tweets_DM.json to pandas dataframe and extract the tweet_id and text part. Second, I merge the dataframe with emotion and data identification go a step further to sperate train/test datasets.

____________________________________________________________

## 2.  Classification

## 2.1 Word2vec model

### Text Preprocess

In this part, I try to do some data preprocessing to increase the cover rate by replacing some tokens to generate a more cleaner data.
1. replace common emojis with adjectives
2. replace @user token with "" token. Also replace #something with "" and "something" two tokens.
3. remove the digits 
4. convert all text into lowercase
5. utilize nltk tweet tokenizer to tokenizer text

### Load pre-train word embeddings model

I use pre-trained glove twitter word embedding vectors to generate embedding matrix.
<br>source :glove.twitter.27B/word2vec.twitter.27B.200d.txt

### Training data preparation

1. Use keras.Tokenizer to tokenize text (num_words=50000).
2. Use keras.texts_to_sequences to convert phrases into sequence numbers.
3. Use keras.pad_sequences to pad the text (maxlen=30).
4. Use LabelEncoder to create one-hot encoding for emotion label.
5. Create word embedding matrix with pre-trained embeddings.
6. Split the data into train set and validation set (test_size=0.2).

### Build model and train with whole data set

I try to find the best setting for inner layer, like BidirectionalLSTM, CuDNNLSTM, LSTM, etc.
<br>And the best result on Kagglge consist of the following layers:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Embedding, LSTM, ReLU, Dropout, Bidirectional ,Flatten #,  Convolution2D
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import RMSprop, Adam

vocab_size = len(tok.word_index) + 1
embedding_layer = Embedding(vocab_size, 
                            EMBEDDING_DIM, 
                            weights=[loaded_embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH, 
                            trainable=False)


model=Sequential()
model.add(embedding_layer)
hidden_nodes = 192
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Flatten())
model.add(Dense(8,activation='sigmoid')) 
model.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy', metrics=['accuracy'])

### Evaluation

In [ ]:
# calculate accuracy

correct_count = 0
for idx, emotion in enumerate(y_pred):
    if emotion == y_val[idx]:
        correct_count += 1
print(len(y_val), len(y_pred))
print(correct_count/len(y_pred))

0.5586353065648048

____________________________________________________________

## 2.2 roBERTa-base pre-trained model

### Preprocessing

1. Convert training data form dataframe to datasets.
2. Use roBERTa-base pre-trained model.
3. Do sentence processing and label processing.

### Training

In [ ]:
from transformers import TrainingArguments, Trainer

OUTPUT_DIR = '/home/nlplab/rola/DM/Roberta3'
LEARNING_RATE = 2e-5
BATCH_SIZE = 8
EPOCH = 1

training_args = TrainingArguments(
    output_dir = OUTPUT_DIR,
    learning_rate = LEARNING_RATE,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    num_train_epochs = EPOCH,
    save_steps=100000,
    # you can set more parameters here if you want
)

# now give all the information to a trainer
trainer = Trainer(
    # set your parameters here
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_val_dataset['train'],
    eval_dataset = train_val_dataset['test'],
    tokenizer = tokenizer
)

### Evaluation

In [ ]:
# calculate accuracy

true_emotion = encoder.inverse_transform(train_val_dataset["test"]["label"])

correct_count = 0
for idx, emotion in enumerate(pred_emotion):
    if emotion == true_emotion[idx]:
        correct_count += 1
        
print(correct_count/len(pred_emotion))

0.6843081552524277